In [1]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 取得系統時間的工具
from datetime import datetime

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

'''
[1] Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
[2] selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
[3] Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
[4] How to select a drop-down menu value with Selenium using Python?
參考網址：https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
'''

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知


In [2]:
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\Amy Wang\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [3]:
# 走訪網址
url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/'

# 縣市及月份
import numpy as np
dates = np.arange(12)
counties = np.arange(22)
    
def visit(): 
    url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/'
    driver.get(url);
    sleep(2)    
    for j in counties :
        # 選擇縣市
        c = Select(driver.find_element(By.CSS_SELECTOR, "select#stationCounty"))
        c.select_by_index(j)
        sleep(1)
        for i in dates :
            # 選擇'月報表'
            t = Select(driver.find_element(By.CSS_SELECTOR, 'select#datatype'))
            t.select_by_visible_text('月報表 (monthly data)')
            sleep(1)
            # 點開月曆
            driver.find_element(By.CSS_SELECTOR, "img.ui-datepicker-trigger").click()
            sleep(0.5)
            # 選擇年份
            yy = Select(driver.find_element(By.CSS_SELECTOR, 'select[data-handler="selectYear"]'))
            yy.select_by_visible_text('2021')
            sleep(1)
            # 選擇月份
            mm = Select(driver.find_element(By.CSS_SELECTOR, 'select[data-handler="selectMonth"]'))
            sleep(0.1)
            mm.select_by_value(f'{i}')
            sleep(0.1)
            # 送出查詢
            driver.find_element(By.CSS_SELECTOR, "img#doquery").click()
            sleep(0.3)
            windows=driver.window_handles
            # 切到新開的查詢分頁
            driver.switch_to.window(windows[1])
            u=driver.current_url
            driver.get(u)
            #測站下拉式選單長度
            w=len(driver.find_elements(By.CSS_SELECTOR, "select#selectStno option"))
            sleep(0.3)
            for y in range(w):
                # 切換該區觀測站
                driver.find_element(By.CSS_SELECTOR, "select#selectStno").click()
                sleep(0.1)
                driver.find_elements(By.CSS_SELECTOR, "select#selectStno option")[y].click()
                sleep(0.1)
                # 下載
                driver.find_element(By.CSS_SELECTOR, "a#downloadCSV").click()
                sleep(0.2)
            # 關閉分頁繼續下個月
            driver.close()
            windows=driver.window_handles
            driver.switch_to.window(windows[0])
            sleep(1)

In [ ]:
# 主程式
if __name__=="__main__":
    visit()   